In [1]:
%matplotlib notebook
from numpy import *
from matplotlib.pyplot import *
from numpy.linalg import qr 
from scipy.linalg import hilbert

In [2]:
def hessenberg(A): 
    m,n = A.shape 
    assert m == n, "A must be square" 
     
    H = A.copy() 
    Qt = eye(m) 
    for j in range(m-1): 
        x = H[j+1:,j:j+1]         
        I = eye(m-j-1) 
        s = 1 if x[0] > 0 else -1     # sign function, with sign(0)  = 1 
        v = s*linalg.norm(x,2)*I[:,0:1] + x 
 
        vn = linalg.norm(v,2) 
        v = v/vn 
        F = I - 2*(v@v.T) 
        H[j+1:,j:] = F@H[j+1:,j:] 
        H[0:,j+1:] = H[0:,j+1:]@F   # Apply F to the right side of H.  
         
    return H 

In [15]:
def WilkinsonShift( a, b, c ):
    # Calculate Wilkinson's shift for symmetric matrices: 
    δ = (a-c)/2
    return c - sign(δ)*b^2/(abs(δ) + sqrt(δ^2+b^2))


def QRwithShifts( A ):
   # The QR algorithm for symmetric A with Rayleigh shifts and Hessenberg reduction. Please use eigvals() in 
   # Julia for serious applications.
    n = size(A,1)
    myeigs = zeros(n)
    if ( n == 1 ):
        myeigs[1] = A[1,1]
    else:
        I = eye( n )
        # Reduction to Hessenberg form:
        A = hessenberg( A )
        # Let's start the shifted QR algorithm with 
        while( linalg.norm(A[n,n-1]) > 1e-10 ):

            mu = WilkinsonShift( A[n-1,n-1], A[n,n], A[n-1,n] )
            # This line should use faster Hessenberg reduction:
            (Q,R) = qr(A - mu*I)
            # This line needs speeding up, currently O(n^3) operations!: 
            A = R*Q + mu*I
       
        # Deflation and recurse:
        myeigs = [A[n,n] , QRwithShifts( A[1:n-1, 1:n-1] )]
   
    return myeigs


In [18]:
H1 = hilbert(5)
#display_mat("Hilbert matrix : ", H1)
H = hessenberg(H1)
